In [ ]:
import kagglehub
import os

dataset_path = kagglehub.dataset_download("shreyaty08/fakeavceleb-extracted-audio")
AUDIO_ROOT = os.path.join(dataset_path, "FakeAVCeleb_Extracted_Audio")
print("Dataset path:", AUDIO_ROOT)

In [ ]:
import librosa
import numpy as np
import glob
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:

SR = 16000
WIN_SEC = 2.0
HOP_SEC = 1.0
N_MELS = 128
N_FFT = 2048
HOP_LEN = 512

INPUT_DIR = "/kaggle/input/fakeavceleb-extracted-audio/FakeAVCeleb_Extracted_Audio"
OUTPUT_DIR = "/content/melspec_windows_fixed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
def audio_to_melspec_fixed(y, sr):
    mel = librosa.feature.melspectrogram(
        y=y,
        sr=sr,
        n_fft=N_FFT,
        hop_length=HOP_LEN,
        n_mels=N_MELS,
        fmin=20,
        fmax=8000
    )

    mel_db = librosa.power_to_db(mel, ref=1.0)


    mel_db = (mel_db - mel_db.mean()) / (mel_db.std() + 1e-8)

    return mel_db

In [ ]:

print("🔄 Processing audio files with FIXED normalization...")

for root, dirs, files in os.walk(INPUT_DIR):
    for file in tqdm(files, desc="Processing"):
        if not file.lower().endswith(".flac"):
            continue

        input_path = os.path.join(root, file)
        rel_path = os.path.relpath(root, INPUT_DIR)
        output_subdir = os.path.join(OUTPUT_DIR, rel_path)
        os.makedirs(output_subdir, exist_ok=True)

        try:
            y, sr = librosa.load(input_path, sr=SR)
        except:
            continue

        win_len = int(WIN_SEC * SR)
        hop_len = int(HOP_SEC * SR)

        min_real = int(0.8 * win_len)
        window_idx = 0

        for start in range(0, len(y), hop_len):
            end = start + win_len
            y_slice = y[start:end]

            if len(y_slice) < win_len:
                if len(y_slice) < min_real:
                    break
                y_slice = np.pad(y_slice, (0, win_len - len(y_slice)))


            mel = audio_to_melspec_fixed(y_slice, SR)

            base = os.path.splitext(file)[0]
            out_file = f"{base}_win{window_idx}.npy"
            save_path = os.path.join(output_subdir, out_file)
            np.save(save_path, mel)

            window_idx += 1

print("✅ Preprocessing Done with Normalzie!")